# Reproduction

This notebook currently imports results from a run of PHC.py and attempts to map them to Table 6.

Table 6 results are from Shoaib M, Ramamohan V. **Simulation modeling and analysis of primary health center operations**. *SIMULATION* 98(3):183-208. (2022). <https://doi.org/10.1177/00375497211030931>.

In [1]:
import xlrd
import pandas as pd

In [2]:
model_path = 'Outputs.xls'
table6_path = '../original_study/tab6.csv'

In [3]:
# Import .xls and convert to pandas dataframe
book = xlrd.open_workbook(model_path)
result = pd.read_excel(book, header=None, index_col=0)
result

,1,2,3,4,5,6,7,8,9,10
0,,,,,,,,,,
OPD patients,32836.000000,33066.000000,33023.000000,32870.000000,33240.000000,33232.000000,33216.000000,33368.000000,33314.000000,33171.000000
IPD patients,174.000000,166.000000,171.000000,177.000000,198.000000,194.000000,206.000000,178.000000,198.000000,178.000000
ANC patients,368.000000,362.000000,386.000000,380.000000,388.000000,381.000000,372.000000,370.000000,350.000000,355.000000
Del patients,384.000000,367.000000,369.000000,361.000000,309.000000,360.000000,359.000000,397.000000,385.000000,339.000000
OPD Q wt,0.006038,0.008066,0.006059,0.006793,0.005244,0.010225,0.007012,0.016915,0.008484,0.018205
Pharmacy Q wt,1.047611,1.019909,1.011980,1.016690,1.006467,1.032219,1.020334,1.017687,1.017410,1.003815
Lab Q wt,2.063080,2.096727,2.121108,1.958464,2.096112,2.160190,2.039157,2.069498,2.135470,2.139755
doc occ,0.269761,0.271455,0.265695,0.271491,0.266211,0.269985,0.269385,0.274394,0.270916,0.264823
Lab patient list,34553.000000,68973.000000,103851.000000,137618.000000,172163.000000,206714.000000,241073.000000,275273.000000,309924.000000,344492.000000


In [4]:
# Find mean from the replications and round to 3dp
av_result = round(result.mean(axis=1), 3)

# Drop duplicate NCD occ
av_result_dup = av_result[~av_result.index.duplicated(keep='first')]

# Calculate the proportion of childbirth cases referred and add to series
prop_del_referred = pd.Series(
    av_result_dup['del referred'] / av_result_dup['Del patients'],
    index=['prop_del_referred'])
new_av_result = pd.concat([av_result, prop_del_referred])

# Convert to df, save as model config1, and display
model_c1 = (new_av_result
            .to_frame(name='config1')
            .reset_index()
            .rename(columns= {0: 'model_output', 'index': 'model_output'}))
model_c1

,model_output,config1
0,OPD patients,33133.600000
1,IPD patients,184.000000
2,ANC patients,371.200000
3,Del patients,363.000000
4,OPD Q wt,0.009000
5,Pharmacy Q wt,1.019000
6,Lab Q wt,2.088000
7,doc occ,0.269000
8,Lab patient list,189463.400000
9,OPD q len,0.009000


In [5]:
# Import table 6
t6 = pd.read_csv(table6_path)
t6_c1 = t6[['outcome', 'config1_mean']]
t6_c1

,outcome,config1_mean
0,Doctor utilisation,0.268
1,NCD Nurse utilisation,0.865
2,Staff nurse utilisation,0.323
3,Pharmacist utilisation,0.643
4,Lab utilisation,0.559
5,Inpatient bed utilisation,0.093
6,Labour bed utilisation,0.283
7,Mean length of OPD queue (number of patients),0.000
8,OPD queue waiting time (minutes),0.009
9,Mean length of pharmacy queue (number of patie...,0.090


In [12]:
# Make dictionary with labels from table 6, and corresponding names from model output
t6_labels = {
  'doc occ': 'Doctor utilisation',
  'NCD occ': 'NCD Nurse utilisation',
  'staff nurse occ': 'Staff nurse utilisation',
  'pharm occ': 'Pharmacist utilisation',
  'lab occ': 'Lab utilisation',
  'ipd bed occ': 'Inpatient bed utilisation',
  'del occ': 'Labour bed utilisation',  # "Del" stands for delivery
  'OPD q len': 'Mean length of OPD queue (number of patients)',
  'OPD Q wt': 'OPD queue waiting time (minutes)',
  'pharmacy q len': 'Mean length of pharmacy queue (number of patients)',
  'Pharmacy Q wt': 'Pharmacy queue waiting time (minutes)',
  'lab q len': 'Mean length of Lab queue (number of patients)',
  'Lab Q wt': 'Lab queue waiting time (minutes)',
  'prop_del_referred': 'Fraction of childbirth cases referred'
}
inv_t6_labels = {v: k for k, v in t6_labels.items()}

In [18]:
# Add labels to model results
model_c1['label'] = model_c1['model_output'].map(t6_labels)

# Merge with table 1
compare = model_c1.merge(
    t6_c1, how='inner', left_on='label', right_on='outcome')
compare

,model_output,config1,label,outcome,config1_mean,model_lab
0,OPD Q wt,0.009000,OPD queue waiting time (minutes),OPD queue waiting time (minutes),0.009,OPD Q wt
1,Pharmacy Q wt,1.019000,Pharmacy queue waiting time (minutes),Pharmacy queue waiting time (minutes),1.025,Pharmacy Q wt
2,Lab Q wt,2.088000,Lab queue waiting time (minutes),Lab queue waiting time (minutes),2.084,Lab Q wt
3,doc occ,0.269000,Doctor utilisation,Doctor utilisation,0.268,doc occ
4,OPD q len,0.009000,Mean length of OPD queue (number of patients),Mean length of OPD queue (number of patients),0.000,OPD q len
5,pharmacy q len,0.090000,Mean length of pharmacy queue (number of patie...,Mean length of pharmacy queue (number of patie...,0.090,pharmacy q len
6,lab q len,0.095000,Mean length of Lab queue (number of patients),Mean length of Lab queue (number of patients),0.094,lab q len
7,NCD occ,0.870000,NCD Nurse utilisation,NCD Nurse utilisation,0.865,NCD occ
8,NCD occ,0.870000,NCD Nurse utilisation,NCD Nurse utilisation,0.865,NCD occ
9,lab occ,0.558000,Lab utilisation,Lab utilisation,0.559,lab occ


No more than 1% difference, except mean OPD queue length comparing 0.009 and 0.000, simply because one value is 0, which basically means, percent difference is undefined for that one.

In [23]:
(compare['config1_mean'] - compare['config1'])/compare['config1']*100

0       0.000000
1       0.588813
2      -0.191571
3      -0.371747
4    -100.000000
5       0.000000
6      -1.052632
7      -0.574713
8      -0.574713
9       0.179211
10      0.000000
11      0.000000
12     -1.393728
13      0.000000
14      1.848921
dtype: float64